Place de marché
==============

![logo](../reports/figures/logo.png)

### Votre mission
Votre mission est de **réaliser une première étude de faisabilité d'un moteur de classification** d'articles basé sur une image et une description pour l'automatisation de l'attribution de la catégorie de l'article.

Pour ce faire, vous allez **évaluer la possibilité d'extraire des données depuis l'API Amazon** en **prenant connaissance de la documentation** et en **écrivant la requête** qui vous permettrait d'extraire des données supplémentaires. Vous vous assurerez ainsi que vous pourrez bien disposer de plus de données et diversifier les sources de données pour éviter les biais pour votre moteur de classification.

Ensuite, vous **analyserez le jeu de données** déjà constitué en **réalisant un prétraitement** des images et des descriptions des produits, une **réduction de dimension**, puis un **clustering**. Les résultats du clustering seront présentés sous la forme d’une représentation en deux dimensions à déterminer, qui ’illustrera le fait que les caractéristiques extraites permettent de regrouper des produits de même catégorie.

La représentation graphique vous aidera à convaincre Linda que cette approche de modélisation permettra bien de regrouper des produits de même catégorie.

### Contraintes

Linda vous a communiqué les contraintes suivantes :

   * Limiter le nombre d’articles pris par l’API (par exemple : 1000 lignes) et filtrer sur un unique type d’article (par exemple un type d’article peu présent dans votre échantillon de données actuelles).
   * Afin d’extraire les features, mettre en œuvre a minima un algorithme de type SIFT / ORB / SURF.
   * Un algorithme de type CNN Transfer Learning peut éventuellement être utilisé en complément, s’il peut apporter un éclairage supplémentaire à la démonstration.

In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from IPython.display import display
import seaborn as sns

sns.set(font_scale=1.5)

## Chargement des données

In [ ]:
for dirname, _, filenames in os.walk('../data/raw/'):
    if len(filenames) == 1:
        df = pd.read_csv(os.path.join(dirname, filenames[0]))

In [ ]:
df.head()

In [ ]:
df['path'] = df['image'].apply(lambda x: os.path.join('../data/raw/Images/', x))

In [ ]:
col_to_drop = [
    'uniq_id',
    'crawl_timestamp',
    'product_url',
    'pid',
    'discounted_price',
    'is_FK_Advantage_product',
    'product_rating',
    'overall_rating',
    'product_specifications',
#     'brand',
    
]
df.drop(columns=col_to_drop, inplace=True)

In [ ]:
df.head()

In [ ]:
df.loc[1, 'product_category_tree']

In [ ]:
df['product_category_tree_len'] = df['product_category_tree'].apply(lambda x: len(x.split('>>')))

In [ ]:
sns.distplot(df['product_category_tree_len'], kde=False)

In [ ]:
df[df['product_category_tree_len'] == 2].loc[873, 'description']

In [ ]:
df[df['product_category_tree_len'] == 2].loc[873, 'product_category_tree']

In [ ]:
def extract_level(tree_str, level=-1, strict=False):
    """return a specific level from product_category_tree.
    tips: specify a negative index to access latest part of the tree.
    """
    tree_str = eval(tree_str)[0]
    levels = tree_str.split('>>')
    levels = list(map(lambda x: x.strip(), levels))
    if strict:
        return levels[level]
    else:
        try:
            return levels[level]
        except IndexError:
            return None

In [ ]:
df['product_category_tree'].apply(extract_level, level=-2)

In [ ]:
_, ax = plt.subplots(1, figsize=(8, 8))

top_10 = df['product_category_tree'].apply(extract_level, level=1).value_counts().iloc[:15].index.tolist()


sns.countplot(y=df['product_category_tree']\
                    .apply(extract_level, level=1)\
                    .apply(lambda x: 'other' if x not in top_10 else x),
              order=df['product_category_tree']\
                    .apply(extract_level, level=1)\
                    .apply(lambda x: 'other' if x not in top_10 else x)\
                    .value_counts().index)

plt.ylabel('Categories')
print('number of modalities : %i' % df['product_category_tree'].apply(extract_level, level=1).value_counts().shape[0])

In [ ]:
_, ax = plt.subplots(1, figsize=(8, 8))

top_10 = df['product_category_tree'].apply(extract_level, level=-2).value_counts().iloc[:15].index.tolist()


sns.countplot(y=df['product_category_tree']\
                    .apply(extract_level, level=-2)\
                    .apply(lambda x: 'other' if x not in top_10 else x),
              order=df['product_category_tree']\
                    .apply(extract_level, level=-2)\
                    .apply(lambda x: 'other' if x not in top_10 else x)\
                    .value_counts().index)

plt.ylabel('Categories')
print('number of modalities : %i' % df['product_category_tree'].apply(extract_level, level=-2).value_counts().shape[0])

In [ ]:
LEVEL = 1

_, ax = plt.subplots(1, figsize=(8, 8))

top_10 = df['product_category_tree'].apply(extract_level, level=LEVEL).value_counts().iloc[:15].index.tolist()


sns.countplot(y=df['product_category_tree']\
                    .apply(extract_level, level=LEVEL)\
                    .apply(lambda x: 'other' if x not in top_10 else x),
              order=df['product_category_tree']\
                    .apply(extract_level, level=LEVEL)\
                    .apply(lambda x: 'other' if x not in top_10 else x)\
                    .value_counts().index)

plt.ylabel('Categories')
print('number of modalities : %i' % df['product_category_tree']\
          .apply(extract_level, level=LEVEL).value_counts().shape[0])

In [ ]:
def scale_down(image, factor=5):
    width, height = image.size
    target_width, target_height = width // factor, height // factor
    return image.resize((target_width, target_height))

images = [scale_down(Image.open(x), factor=5) for x in df['path']]

In [ ]:
size = 10
f, axes = plt.subplots(size, size, figsize=(12, 12))
for ax, im in zip(axes.flatten(), random.sample(images, size ** 2)):
    ax.imshow(im, cmap='gray', aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])  # to hide tick values on X and Y axis

In [ ]:
df['size'] = df['path'].apply(lambda x: Image.open(x).size)

In [ ]:
df['size']

In [ ]:
df['width'] = df['size'].apply(lambda x: x[0])

In [ ]:
df['height'] = df['size'].apply(lambda x: x[1])

In [ ]:
def distplot(series, **kwargs):
    """Create a figure with two subplots.
    The lower part of the figure is distplot and the upper part display
    a box plot for the same sample.

    :arg:
        series (pd.Series): The sample you want to plot.
        kwargs : all keyword argument accepted by seaborn.distplot.
    """
    # Cut the window in 2 parts
    kwrgs = {"height_ratios": (.15, .85)}
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(8, 8),
                                        gridspec_kw=kwrgs)

    # Add a graph in each part
    sns.boxplot(series, ax=ax_box)
    sns.distplot(series, ax=ax_hist, **kwargs)

    # Remove x axis name for the boxplot
    ax_box.set(xlabel='')
    return ax

In [ ]:
distplot(df['width'])

In [ ]:
distplot(df['height'])

In [ ]:
df['width'].describe()

In [ ]:
df['height'].describe()

In [ ]:
df['nb_pix'] = df['width'] * df['height']

In [ ]:
df['nb_pix'].describe()

In [ ]:
df[df['nb_pix'] == df['nb_pix'].max()]

In [ ]:
count = df['product_category_tree'].apply(extract_level, level=0).value_counts()

In [ ]:
count